Я разобрался с алгоритмом,но даже со словарём,в котором около 1000 слов (и соответсвенно в матрице примерно 300к не 0 элементов) программа работает оооочень долго. При том,что для стабилизации нужно хотя бы пара тысяч итераций,а проход по 1000 элементам матрицы занимает чуть меньше секунды,то получается,что не 0 элементов должно быть ~1-2к.что очень мало,честно,не знаю как написать код алгоритма оптимально. Т.к. сеиминара не было,решил отправитьвам PR ,можете,пожалуйста,подсказать : как следует оптимизировать код алгоритма?м 

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

In [3]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
print(len(newsgroups_train.data))

11314


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True,max_df=0.9,min_df=0.01)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(binary=True, max_df=0.9, min_df=0.01,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

In [6]:
X_train1 = vectorizer.transform(newsgroups_train.data)
print(type(X_train1))
X_train1.shape

<class 'scipy.sparse.csr.csr_matrix'>


(11314, 1142)

In [7]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
print(type(X_train))
X_train.shape
a=X_train1.nonzero()
a=np.transpose(a)
print(len(a))

<class 'scipy.sparse.csr.csr_matrix'>
313237


Ну,содержательная часть :

In [10]:
const=50
Ntopics=20
alpha=1/Ntopics
beta=const/X_train.shape[1]
z=np.zeros((X_train.shape[0],X_train.shape[1]),dtype = np.int8)
Nkw=np.zeros((Ntopics,X_train.shape[1]),dtype = np.int32)
Ndk=np.zeros((Ntopics,X_train.shape[0]),dtype = np.int16)
Nk=np.zeros(Ntopics,dtype = np.int32)
for i in a :
    c=np.random.randint(Ntopics)
    Nkw[c,i[1]]+=1
    Nk[c]+=1
    #print(Nk)
    Ndk[c,i[0]]+=1
    z[i[0],i[1]]=c
print(Nk)

[15666 15847 15495 15790 15744 15623 15569 15558 15668 15719 15877 15577
 15683 15653 15619 15840 15522 15620 15442 15725]


In [40]:
import time
def lda(X_train,Nkw1,Ndk1,Nk1,z1) :
    Ndk=Ndk1
    Nkw=Nkw1
    Nk=Nk1
    z=z1
    P=np.zeros(Ntopics,dtype = np.double)
    a=X_train.nonzero()
    a=np.transpose(a)
    print("кол-во не 0 эл-ов")
    print(len(a))
    for j in range(400) :
        print(j)
        start_time = time.time()
        for i in a:
            #gg+=1
            #if gg%1000==0 : 
                #print(gg)
            top=z[i[0],i[1]]
            i0=i[0]
            i1=i[1]
            Ndk[top,i0]-=1
            Nk[top]-=1
            Nkw[top,i1]-=1
            for k in range(Ntopics) :
                P[k]=(Ndk[k,i0]+alpha)*(Nkw[k,i1]+beta)/(Nk[k]+const)
            P=np.cumsum(P)
            c=np.random.rand()*P[-1]
            for t in range(Ntopics) :
                if P[t] > c : 
                    z[i0,i1]=t 
                    break
            Ndk[t,i0]+=1
            Nk[t]+=1
            Nkw[t,i1]+=1
        print("время на 1 итерацию")
        print(time.time() - start_time)
    return z,Ndk,Nkw
z1=np.zeros((X_train.shape[0],X_train.shape[1]),dtype = np.int8)
Nkw1=np.zeros((Ntopics,X_train.shape[1]),dtype = np.int8)
Ndk1=np.zeros((Ntopics,X_train.shape[0]),dtype = np.int8)
Nk1=np.zeros(Ntopics,dtype = np.int32)
X_trainn=X_train[:100,:]
z1,Ndk1,Nkw1=lda(X_train,Nkw,Ndk,Nk,z)
print(Ndk1[:,0])

кол-во не 0 эл-ов
313237
0
время на 1 итерацию
78.36442542076111
1
время на 1 итерацию
72.98662257194519
2
время на 1 итерацию
72.22908639907837
3
время на 1 итерацию
69.03182625770569
4
время на 1 итерацию
67.04641938209534
5
время на 1 итерацию
76.8603618144989
6
время на 1 итерацию
67.1655068397522
7
время на 1 итерацию
77.85406422615051
8
время на 1 итерацию
90.16276931762695
9
время на 1 итерацию
105.98095989227295
10
время на 1 итерацию
85.56452250480652
11
время на 1 итерацию
87.08459401130676
12
время на 1 итерацию
83.38998103141785
13
время на 1 итерацию
90.0006537437439
14
время на 1 итерацию
88.34848809242249
15
время на 1 итерацию
86.31805038452148
16
время на 1 итерацию
101.46676659584045
17
время на 1 итерацию
81.163405418396
18
время на 1 итерацию
97.81419682502747
19
время на 1 итерацию
92.37932348251343
20
время на 1 итерацию
87.4678635597229
21
время на 1 итерацию
88.64669823646545
22
время на 1 итерацию
87.18866729736328
23
время на 1 итерацию
85.90175533294678
24
вр

время на 1 итерацию
69.12590599060059
199
время на 1 итерацию
75.53042221069336
200
время на 1 итерацию
71.78675961494446
201
время на 1 итерацию
75.9026849269867
202
время на 1 итерацию
74.10441040992737
203
время на 1 итерацию
72.21907925605774
204
время на 1 итерацию
71.79678153991699
205
время на 1 итерацию
76.22391128540039
206
время на 1 итерацию
74.07839345932007
207
время на 1 итерацию
69.96548581123352
208
время на 1 итерацию
75.76258778572083
209
время на 1 итерацию
76.98544836044312
210
время на 1 итерацию
75.54743313789368
211
время на 1 итерацию
76.53813195228577
212
время на 1 итерацию
73.30384850502014
213
время на 1 итерацию
73.15975737571716
214
время на 1 итерацию
80.7641088962555
215
время на 1 итерацию
75.79761147499084
216
время на 1 итерацию
70.9421739578247
217
время на 1 итерацию
74.92399454116821
218
время на 1 итерацию
77.08351850509644
219
время на 1 итерацию
72.17304635047913
220
время на 1 итерацию
74.97502851486206
221
время на 1 итерацию
71.59964108467102

время на 1 итерацию
64.2104139328003
395
время на 1 итерацию
62.69634819030762
396
время на 1 итерацию
62.23001050949097
397
время на 1 итерацию
61.634594202041626
398
время на 1 итерацию
61.07219576835632
399
время на 1 итерацию
60.92208790779114
[0 0 0 0 0 0 8 0 2 6 0 0 0 0 7 0 0 0 0 0]


In [44]:
for i in range(Ntopics):
  mask = (Nkw1[i] >= np.sort(Nkw1[i,:])[-10])
  print("%d Topic: {" % (i + 1), " ".join(vectorizer.inverse_transform(mask)[0]),"}")

1 Topic: { don edu good just know like people soon think ve }
2 Topic: { article com don edu just know mail post posted read }
3 Topic: { believe bible christ christian christians does god jesus people think }
4 Topic: { good high just low power time use used using way }
5 Topic: { don just know like people really thing things think ve }
6 Topic: { federal government gun law laws people public rights state states }
7 Topic: { bike car cars don good got just like right think }
8 Topic: { better don game good just like players team think year }
9 Topic: { different does don example fact know people point say think }
10 Topic: { advance appreciated does help hi information know looking mail thanks }
11 Topic: { chip clipper encryption government key keys phone public secure use }
12 Topic: { asking best condition edu interested new offer price sale sell shipping }
13 Topic: { government israel israeli jewish jews like people right state war }
14 Topic: { game games good just new play seas

In [45]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
Попробуем соотнести некоторые топики :
1)тег  'soc.religion.christian' хорошо соотносится с Топиком №3:
    { believe bible christ christian christians does god jesus people think }
2)тег 'talk.politics.guns' хорошо соотносится с Топиком №6:
    { federal government gun law laws people public rights state states } 
3)тег 'misc.forsale'   хорошо соотносится с Топиком №12:
    { best condition edu interested new offer price sale sell shipping }
4)тег 'comp.windows.x'   хорошо соотносится с Топиком №18
    { does file files like problem program use using window windows }
5)тег 'rec.sport.hockey'   хорошо соотносится с Топиком №14:
    { game games good like play players season team think year }
5)тег 'comp.graphics'   хорошо соотносится с Топиком №19:
    { card disk does drive just know problem use video work }